In [1]:
import asyncio

In [4]:
async def agent():
    print('[agent] start')
    for i in range(10):
        await asyncio.sleep(1)
        print('[agent]', i)
    print('[agent] end')
    return 1

In [6]:
async def guardrail():
    print('[guardrail] start')
    await asyncio.sleep(1.5)
    print('[guardrail] check pass')
    print('[guardrail] stop')
    return 1

In [7]:
await guardrail()
await agent()

[guardrail] start
[guardrail] check pass
[guardrail] stop
[agent] start
[agent] 0
[agent] 1
[agent] 2
[agent] 3
[agent] 4
[agent] 5
[agent] 6
[agent] 7
[agent] 8
[agent] 9
[agent] end


1

In [8]:
check_pass = await guardrail()
if check_pass:
    await agent()

[guardrail] start
[guardrail] check pass
[guardrail] stop
[agent] start
[agent] 0
[agent] 1
[agent] 2
[agent] 3
[agent] 4
[agent] 5
[agent] 6
[agent] 7
[agent] 8
[agent] 9
[agent] end


In [9]:
r1, r2 = await asyncio.gather(guardrail(), agent()) # parallel

[guardrail] start
[agent] start
[agent] 0
[guardrail] check pass
[guardrail] stop
[agent] 1
[agent] 2
[agent] 3
[agent] 4
[agent] 5
[agent] 6
[agent] 7
[agent] 8
[agent] 9
[agent] end


In [12]:
class GuardrailException(Exception):
    pass

async def guardrail_fail():
    print('[guardrail] start')
    await asyncio.sleep(1.5)
    print('[guardrail] check fails')
    raise GuardrailException("check fails")

In [13]:
r1, r2 = await asyncio.gather(guardrail_fail(), agent()) # parallel

[guardrail] start
[agent] start
[agent] 0
[guardrail] check fails


GuardrailException: check fails

[agent] 1
[agent] 2
[agent] 3
[agent] 4
[agent] 5
[agent] 6
[agent] 7
[agent] 8
[agent] 9
[agent] end


In [16]:
try:
    agent_task = asyncio.create_task(agent())
    guardrail_task = asyncio.create_task(guardrail_fail())

    r1, r2 = await asyncio.gather(agent_task, guardrail_task) # parallel

except GuardrailException as e:
    agent_task.cancel()

    try:
        await agent_task
    except asyncio.CancelledError:
        print('[main] agent cancelled')


[agent] start
[guardrail] start
[agent] 0
[guardrail] check fails
[main] agent cancelled


In [17]:
async def run_with_guardrails(agent_coroutine, guardrails):
    """
    Run `agent_coroutine` while multiple guardrails monitor it.

    Parameters:
        agent_coroutine: an *awaitable*, e.g. agent()
        guardrails: an iterable of *awaitables*, e.g. [guard1(), guard2()]

    Returns:
        The result of the agent, if no guardrail triggers.

    Raises:
        GuardrailException from any guardrail.
    """

    agent_task = asyncio.create_task(agent_coroutine)
    guard_tasks = [asyncio.create_task(g) for g in guardrails]

    try:
        # If any guardrail raises GuardrailException,
        # gather will throw and we drop into except.
        await asyncio.gather(agent_task, *guard_tasks)

        # Agent finished successfully.
        return agent_task.result()

    except GuardrailException as e:
        # At least one guardrail fired.
        print("[guardrail fired]", e.info)

        # Cancel the agent.
        agent_task.cancel()
        try:
            await agent_task
        except asyncio.CancelledError:
            print("[run_with_guardrails] agent cancelled")

        # Cancel all guardrails (they may still be running).
        for t in guard_tasks:
            t.cancel()
        await asyncio.gather(*guard_tasks, return_exceptions=True)

        raise


In [18]:
result = await run_with_guardrails(
    agent(),
    [guardrail()]
)

[agent] start
[guardrail] start
[agent] 0
[guardrail] check pass
[guardrail] stop
[agent] 1
[agent] 2
[agent] 3
[agent] 4
[agent] 5
[agent] 6
[agent] 7
[agent] 8
[agent] 9
[agent] end


In [20]:
from dataclasses import dataclass

@dataclass
class GuardrailFunctionOutput:
    output_info: str
    tripwire_triggered: bool


In [21]:
class GuardrailException(Exception):
    def __init__(self, message: str, info: GuardrailFunctionOutput):
        super().__init__(message)
        self.info = info


In [22]:
import ver3_asyncio_guardrail

In [23]:
import search_agent

In [24]:
agent = search_agent.create_agent()

In [ ]:
result = await run_with_guardrails(
    ver3_asyncio_guardrail.run(agent, 'llm as a judge'),
    [guardrail(), guardrail_fail()]
)

[guardrail] start
[guardrail] start
TOOL CALL (search): input_guardrail({"message":"llm as a judge"})
[guardrail] check pass
[guardrail] stop
[guardrail] check fails


TypeError: GuardrailException.__init__() missing 1 required positional argument: 'info'

# Invalid Query



## Validation Error

The input query contained topics that may violate guidelines or were insufficiently detailed for processing. Please provide a clearer and more specific question regarding Evidently or its features.

### References



## References



In [27]:
result = await run_with_guardrails(
    ver3_asyncio_guardrail.run(agent, 'llm as a judge'),
    [guardrail()]
)

[guardrail] start
[guardrail] check pass
[guardrail] stop
TOOL CALL (search): input_guardrail({"message":"llm as a judge"})
# Input Error



## Input Validation Failed

Your query contains prohibited topics and could not be processed.

### References



## References

